In [1]:
import findspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .appName("Домашнее задание 2") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
#Выберите 15 стран с наибольшим процентом переболевших на 31 марта 
#(в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

"""
Задание 1 - здесь вы вывели просто значение колонки total_cases - но эта колонка означает количество зафиксированных
на данный день случаев (суммарно), а не процент переболевших. Чтобы посчитать процент переболевших, нужно подумать,
из каких колонок это можно вывести. Самое простое - взять total_cases и поделить на population - уже будет неплохая оценка
процента переболевших. Только не забудьте умножить на 100.

"""
"""
Давайте в 1 и 2 заданиях поменяем год на 2021 тоже и посмотрим на результаты.
"""


'\nДавайте в 1 и 2 заданиях поменяем год на 2021 тоже и посмотрим на результаты.\n'

In [5]:
from pyspark.sql.types import *
import pyspark.sql.functions as func

In [6]:
schema = StructType([StructField("iso_code", StringType(), False),
                     StructField("continent", StringType(), True),
                     StructField("location", StringType(), False),
                     StructField("date", TimestampType(), False),
                     StructField("total_cases", IntegerType(), True),
                     StructField("population", IntegerType(), True)
                    ])

In [7]:
df = (spark.read
          .option('header', True)
          .option('sep', ',')
          .option('inferSchema', True)
          .csv('Downloads/owid-covid-data.csv')
     )

In [8]:
df = df.select('iso_code', 'location', 'total_cases', "population") \
.filter((df.date == '2021-03-31') & (df.continent.isNotNull()))


In [9]:
df = df.withColumn('Result', func.round(df['total_cases'] / df['population'] * 100,2))

In [10]:
df.sort(df.Result.desc()).show(15)

+--------+-------------+-----------+------------+------+
|iso_code|     location|total_cases|  population|Result|
+--------+-------------+-----------+------------+------+
|     AND|      Andorra|    12010.0|     77265.0| 15.54|
|     MNE|   Montenegro|    91218.0|    628062.0| 14.52|
|     CZE|      Czechia|  1532332.0| 1.0708982E7| 14.31|
|     SMR|   San Marino|     4730.0|     33938.0| 13.94|
|     SVN|     Slovenia|   215602.0|   2078932.0| 10.37|
|     LUX|   Luxembourg|    61642.0|    625976.0|  9.85|
|     ISR|       Israel|   833105.0|   8655541.0|  9.63|
|     USA|United States| 3.046221E7|3.31002647E8|   9.2|
|     SRB|       Serbia|   600596.0|   6804596.0|  8.83|
|     BHR|      Bahrain|   144445.0|   1701583.0|  8.49|
|     PAN|       Panama|   355051.0|   4314768.0|  8.23|
|     PRT|     Portugal|   821722.0| 1.0196707E7|  8.06|
|     EST|      Estonia|   106424.0|   1326539.0|  8.02|
|     SWE|       Sweden|   804886.0|  1.009927E7|  7.97|
|     LTU|    Lithuania|   2161

In [11]:
'''
Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном 
порядке по убыванию (в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

Задание 2 - здесь вы используете неправильную колонку для выполнения вычислений. 
Total_cases - это суммарное количество случаев, зафиксированных К данной дате. Оно всегда увеличивается, 
поэтому когда вы взяли max(total_cases) - у вас всегда максимум будет 31-го числа (самая поздняя дата). 
Чтобы вычислить правильно, в этом задании нужно использовать колонку new_cases - это количество новых случаев, 
зафиксированных В данную дату.
'''
"""
Давайте в 1 и 2 заданиях поменяем год на 2021 тоже и посмотрим на результаты.
"""



'\nДавайте в 1 и 2 заданиях поменяем год на 2021 тоже и посмотрим на результаты.\n'

In [12]:
schema = StructType([StructField("iso_code", StringType(), False),
                     StructField("continent", StringType(), True),
                     StructField("location", StringType(), False),
                     StructField("date", TimestampType(), False),
                     StructField("new_cases", IntegerType(), True)
                    ])

In [13]:
df = (spark.read
          .option('header', True)
          .option('sep', ',')
          .option('inferSchema', True)
          .csv('Downloads/owid-covid-data.csv')
     )

In [14]:
df.select('iso_code', 'location', 'new_cases', 'date') \
.filter((df.date >= '2021-03-24') &  (df.date <= '2021-03-31') & (df.continent.isNotNull())) \
.createOrReplaceTempView("hmmm")

In [15]:
spark.sql("Select t2.date, t1.* " +
          "from (select iso_code, location, max(new_cases) mx_new_cases " +
          "from hmmm " +
          "group by iso_code, location) t1 left join hmmm t2 on t1.iso_code = t2.iso_code and t1.mx_new_cases = t2.new_cases " +
          "order by t1.mx_new_cases desc").show(15)

+----------+--------+-------------+------------+
|      date|iso_code|     location|mx_new_cases|
+----------+--------+-------------+------------+
|2021-03-25|     BRA|       Brazil|    100158.0|
|2021-03-24|     USA|United States|     86960.0|
|2021-03-31|     IND|        India|     72330.0|
|2021-03-24|     FRA|       France|     65392.0|
|2021-03-31|     TUR|       Turkey|     39302.0|
|2021-03-26|     POL|       Poland|     35145.0|
|2021-03-31|     DEU|      Germany|     25014.0|
|2021-03-26|     ITA|        Italy|     24076.0|
|2021-03-25|     PER|         Peru|     19206.0|
|2021-03-26|     UKR|      Ukraine|     18226.0|
|2021-03-30|     SWE|       Sweden|     16427.0|
|2021-03-31|     ARG|    Argentina|     16056.0|
|2021-03-29|     ESP|        Spain|     15501.0|
|2021-03-26|     HUN|      Hungary|     11265.0|
|2021-03-24|     CZE|      Czechia|     10972.0|
+----------+--------+-------------+------------+
only showing top 15 rows



In [16]:
'''
Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. 
(например: в россии вчера было 9150 , сегодня 8763, итог: -387) 
(в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, 
кол-во новых случаев сегодня, дельта)

Задание 3 - здесь вычисляете верно, но не за тот год - нужно за 2021. 
И еще можно использовать функцию lag - это то же самое, что lead с параметром -1
'''
"""
В задании 3 тоже все верно, но записей для 24-го марта и 1 апреля в результате быть не должно.
Придумайте, как их отфильтровать. Но при этом в результатах не должно быть 0 или null.
"""


'\nВ задании 3 тоже все верно, но записей для 24-го марта и 1 апреля в результате быть не должно.\nПридумайте, как их отфильтровать. Но при этом в результатах не должно быть 0 или null.\n'

In [17]:
schema = StructType([StructField("iso_code", StringType(), False),
                     StructField("continent", StringType(), True),
                     StructField("location", StringType(), False),
                     StructField("date", TimestampType(), False),
                     StructField("new_cases", IntegerType(), True)
                    ])

In [18]:
df = (spark.read
          .option('header', True)
          .option('sep', ',')
          .option('inferSchema', True)
          .csv('Downloads/owid-covid-data.csv')
     )

In [19]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag 

In [20]:
windowSpec  = Window.partitionBy("iso_code").orderBy("date")

In [21]:
ddf = df.select('iso_code', 'location', 'date', 'new_cases') \
.filter((df.iso_code == 'RUS') & (df.date >= '2021-03-23') &  (df.date <= '2021-03-31')) \
.withColumn("lag",lag("new_cases", 1).over(windowSpec)).na.fill(value=0)

In [22]:
ddf = ddf.withColumn('Result', ( ddf['new_cases'] - ddf['lag']))

In [23]:
ddf = ddf.sort(ddf.date.desc()).limit(7)

In [24]:
ddf.sort(ddf.date).show()

+--------+--------+----------+---------+------+------+
|iso_code|location|      date|new_cases|   lag|Result|
+--------+--------+----------+---------+------+------+
|     RUS|  Russia|2021-03-25|   9128.0|8769.0| 359.0|
|     RUS|  Russia|2021-03-26|   9073.0|9128.0| -55.0|
|     RUS|  Russia|2021-03-27|   8783.0|9073.0|-290.0|
|     RUS|  Russia|2021-03-28|   8979.0|8783.0| 196.0|
|     RUS|  Russia|2021-03-29|   8589.0|8979.0|-390.0|
|     RUS|  Russia|2021-03-30|   8162.0|8589.0|-427.0|
|     RUS|  Russia|2021-03-31|   8156.0|8162.0|  -6.0|
+--------+--------+----------+---------+------+------+

